# Week Five Class One: Covering Problem (Trail Mix)

## Table of Contents<a id="TOC"></a>

1. [Install Packages](#S1) <br>
2. [Trail Mix Problem (LP Covering Model)](#S2) <br>
3. [VERSION ONE: Create a Pyomo model by entering all the values](#S3) <br>
4. [VERSION TWO: Create a Pyomo model by reading in data from Excel](#S4) <br>
5. [ERRORS](#S5) <br>

## 1. Install Packages<a id=S1></a>

In [2]:
import pandas as pd
import pyomo.environ as pe

##### [Back to Top](#TOC)

## 2. Trail Mix Problem (LP Covering Model)<a id=S2></a>

We are going to solve the trail mix (covering) problem that we looked at this problem back in week one or two. The problem has the following decision variables:

$x_0$ = Seeds,
$x_1$ = Raisins,
$x_2$ = Flakes,
$x_3$ = Pecans,
$x_4$ = Walnuts

The full linear programming problem is given next. Recall that the constraints are related to Vitamins, Minerals, Protein and Calories.

\begin{array}{ll}
  \min          &   4 x_0 +   5 x_1 +   3 x_2 +   7 x_3 +   6 x_4\\
  \mathrm{s.t.} &  10 x_0 +  20 x_1 +  10 x_2 +  30 x_3 +  20 x_4\geq 16\\
                &   5 x_0 +   7 x_1 +   4 x_2 +   9 x_3 +   2 x_4\geq 10\\
                &   1 x_0 +   4 x_1 +  10 x_2 +   2 x_3 +   1 x_4\geq 15\\
                & 500 x_0s + 450 x_1 + 160 x_2 + 300 x_3 + 500 x_4\geq 600\\
                & x_0, x_1, x_2, x_3, x_4\geq 0
\end{array}

##### [Back to Top](#TOC)

## 3. VERSION ONE: Create a Pyomo model by entering all the values<a id=S3></a>
In this version of the model, we will enter all the model components "by hand" and we will use generic references to our decision variables as $x_0$, $x_1$, etc

## Create pyomo model, decision variables, objective function, and constraints

In this section you will instantiate the decision variables, objective function and constraints for the Pyomo model.

First we will instantiate a ConcreteModel and store it in the variable `model`. This assumes we have already run `import pyomo.environ as pe`.

In [12]:
model = pe.ConcreteModel()

In [13]:
## Does not exist yet
model.obj

AttributeError: 'ConcreteModel' object has no attribute 'obj'

#### Create Decision Variables

Create a list called `DV_indexes` containing indexes for the decision variables ranging from 0 to 4 for the amount of seeds, raisins, flakes, pecans, and walnuts. Note the range(stop before) command below will generate [0,1,2,3,4].

In [7]:
DV_indexes = list(range(5))
DV_indexes

[0, 1, 2, 3, 4]

Create a pyomo variable named `x` with domain of nonnegative of real numbers.  Make sure you "attach" this variable to the `model` object. For this model x[0] = number of seeds, x[1] = number of raisins, and x[2] = number of flakes.

In [14]:
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals)

To double check what you entered, use '.pprint()'.

In [15]:
model.x.pprint()

x : Size=5, Index=x_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :  None : False :  True : NonNegativeReals
      1 :     0 :  None :  None : False :  True : NonNegativeReals
      2 :     0 :  None :  None : False :  True : NonNegativeReals
      3 :     0 :  None :  None : False :  True : NonNegativeReals
      4 :     0 :  None :  None : False :  True : NonNegativeReals


In [16]:
print(model.x)

x


#### Create the Objective Function

The next cell defines the objective function `obj` using the data in the `coef` dataframe.  The expression argument, `expr=...`, shows a shorthand way to represent a sumproduct using a "list comprehension". The sense=pe.minimize means to minimize the objective function. Note you will find lots of code with the `sense=-1` argument which also means to maximize (`sense=1` is minimization, the default).

In [17]:
model.obj = pe.Objective(expr = 4*model.x[0] + 5*model.x[1] + 3*model.x[2] + 
                         7*model.x[3] + 6*model.x[4], 
                         sense = pe.minimize)

To double check what you entered, use 'pprint'.

In [18]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 4*x[0] + 5*x[1] + 3*x[2] + 7*x[3] + 6*x[4]


#### Create the constraints

Next we need to define the four constraints using the data in the `coef` and `rhs` dataframes.  Make sure to "attach" each constraint to the `model` object. The constraints use a list comprehension to calculate the LHS sumproduct. 

In [19]:
model.cons_vit = pe.Constraint(expr=10*model.x[0] + 20*model.x[1] + 10*model.x[2] + 
                               30*model.x[3] + 20*model.x[4] >= 16)
model.cons_min = pe.Constraint(expr = 5*model.x[0] + 7*model.x[1] + 4*model.x[2] + 
                               9*model.x[3] + 2*model.x[4] >= 10)
model.cons_prot = pe.Constraint(expr = model.x[0] + 4*model.x[1] + 10*model.x[2] + 
                                2*model.x[3] + 1*model.x[4] >= 15)
model.cons_cal = pe.Constraint(expr = 500*model.x[0] + 450*model.x[1] + 160*model.x[2] + 
                               300*model.x[3] + 500*model.x[4] >= 600)

Check the constraints. __NOTE:__ Lower and Upper shows the allowable range. The Upper is the RHS which is +Inf for all of these >= RHS constraints. Note for the first one for vitamins has Lower=16 and Upper=+Inf.

In [20]:
model.cons_vit.pprint()
model.cons_min.pprint()
model.cons_prot.pprint()
model.cons_cal.pprint()

cons_vit : Size=1, Index=None, Active=True
    Key  : Lower : Body                                            : Upper : Active
    None :  16.0 : 10*x[0] + 20*x[1] + 10*x[2] + 30*x[3] + 20*x[4] :  +Inf :   True
cons_min : Size=1, Index=None, Active=True
    Key  : Lower : Body                                       : Upper : Active
    None :  10.0 : 5*x[0] + 7*x[1] + 4*x[2] + 9*x[3] + 2*x[4] :  +Inf :   True
cons_prot : Size=1, Index=None, Active=True
    Key  : Lower : Body                                    : Upper : Active
    None :  15.0 : x[0] + 4*x[1] + 10*x[2] + 2*x[3] + x[4] :  +Inf :   True
cons_cal : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                 : Upper : Active
    None : 600.0 : 500*x[0] + 450*x[1] + 160*x[2] + 300*x[3] + 500*x[4] :  +Inf :   True


For a final check, let's print the entire model (rather than each separate part obj, decision variables, constraints)


In [21]:
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*x[0] + 5*x[1] + 3*x[2] + 7*x[3] + 6*x[4]

4 Constraint Declarations
    cons_cal : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                 : Upper : Active
  

#### Solve the Model

Run the solver and examine the solution. You want to see "OPTIMAL LP Solution Found"

In [22]:
opt = pe.SolverFactory('glpk')
success = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpoqt7hknl.glpk.raw
 --wglp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpafv2spg7.glpk.glp
 --cpxlp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp_g2844md.pyomo.lp
Reading problem data from '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp_g2844md.pyomo.lp'...
5 rows, 6 columns, 21 non-zeros
54 lines were read
Writing problem data to '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpafv2spg7.glpk.glp'...
45 lines were written
GLPK Simplex Optimizer, v4.65
5 rows, 6 columns, 21 non-zeros
Preprocessing...
4 rows, 5 columns, 20 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  5.000e+02  ratio =  5.000e+02
GM: min|aij| =  4.229e-01  max|aij| =  2.364e+00  ratio =  5.590e+00
EQ: min|aij| =  1.789e-01  max|aij| =  1.000e+00  ratio =  5.590e+00
Constructing initial basis...
Size of triangular part is 4
      0: obj =   0.000000

This next cell will help you determine whether the solver found a solution or had an error (like poorly defined constraints, or infeasibility).  Pay particular attention to the "Solver" part of the output.

In [23]:
print(success)


Problem: 
- Name: unknown
  Lower bound: 7.53579952267303
  Upper bound: 7.53579952267303
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of nonzeros: 21
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.013626813888549805
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



#### View the Final Model Obj Function and Dec Variables

The next cell will extract the objective function and optimal solution (assuming the solver found a solution).  

In [25]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

# Get a list of optimal decision variable values with a loop
print('optimal DV')
for index in DV_indexes:
    print(index,round(model.x[index].value,3))

optimal objective value minimum cost = $7.54
optimal DV
0 0.477
1 0.334
2 1.319
3 0.0
4 0.0


Another list comprehension:

In [26]:
new_list = [round(model.x[i].value, 3) for i in DV_indexes]
new_list

[0.477, 0.334, 1.319, 0.0, 0.0]

#### View the slack

Print the slack in each constraint.   __NOTE:__ that ?.? e-13 is basically 0.

In [27]:
print(f'Vitamins slack = {model.cons_vit.slack():.2f}')
print(f'Minerals slack = {model.cons_min.slack():.2f}')
print(f'Protein slack = {model.cons_prot.slack():.2f}')
print(f'Calories slack = {model.cons_cal.slack():.2f}')

Vitamins slack = 8.64
Minerals slack = 0.00
Protein slack = 0.00
Calories slack = 0.00


Also examine the final values for the LHS for each constraint. The "Body" is the final value.

In [28]:
model.cons_vit.display()
model.cons_min.display()
model.cons_prot.display()
model.cons_cal.display()

cons_vit : Size=1
    Key  : Lower : Body              : Upper
    None :  16.0 : 24.64200477326973 :  None
cons_min : Size=1
    Key  : Lower : Body               : Upper
    None :  10.0 : 10.000000000000016 :  None
cons_prot : Size=1
    Key  : Lower : Body               : Upper
    None :  15.0 : 15.000000000000039 :  None
cons_cal : Size=1
    Key  : Lower : Body              : Upper
    None : 600.0 : 600.0000000000007 :  None


##### [Back to Top](#TOC)

## 4. VERSION TWO: Create a Pyomo model by reading in data from Excel<a id=S4></a>

Since we already have all the data in Excel, there is no reason to hand type those in. So here we will read in the needed data from the **Covering** tab in the worksheet **W4L2CoveringAllocationProductMix.xlsx**.



\begin{array}{ll}
  \min          &   4 seeds +   5 raisins +   3 flakes +   7 pecans +   6 walnuts\\
  \mathrm{s.t.} &  10 seeds +  20 raisins +  10 flakes +  30 pecans +  20 walnuts\geq 16\\
                &   5 seeds +   7 raisins +   4 flakes +   9 pecans +   2 walnuts\geq 10\\
                &   1 seeds +   4 raisins +  10 flakes +   2 pecans +   1 walnuts\geq 15\\
                & 500 seeds + 450 raisins + 160 flakes + 300 pecans + 500 walnuts\geq 600\\
                & seeds, raisins, flakes, pecans, walnuts\geq 0
\end{array}

In [31]:
raw_data = pd.read_excel("w05-c01-covering-allocation.xlsx", sheet_name = "Covering")
type(raw_data)

pandas.core.frame.DataFrame

In [33]:
pd.read

AttributeError: module 'pandas' has no attribute 'read'

In [32]:
raw_data

,Covering: Trail Mix,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Decision Variables,x0,x1,x2,x3,x4,NaN,NaN,NaN,NaN
2,NaN,Seeds,Raisins,Flakes,Pecans,Walnuts,NaN,NaN,NaN,NaN
3,Amounts,0.2,0.2,0.2,0.2,0.2,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Objective Function,NaN,NaN,NaN,NaN,NaN,Total,NaN,NaN,NaN
6,Cost,4,5,3,7,6,5.0,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Constraints,NaN,NaN,NaN,NaN,NaN,LHS,NaN,RHS,slack
9,Vitamins,10,20,10,30,20,18,>=,16,-2


Now, rather than some 'generic' decision variables x_0, x_1, etc, let's use the actual decision variable names

In [34]:
DV_indexes = ['seeds', 'raisins', 'flakes', 'pecans', 'walnuts']

This next cell extracts the coefficients of the objective function (cost) and the LHS of the ingredient constraints. Remember that indexes start with 0. So the first row is 0 and the first column is 0. In addition, remember that ranges go "up to" the upper limit so 1:6 is 1,2,3,4,5. Defining the indexes defines the row names.

In [36]:
coef = pd.DataFrame(raw_data.iloc[[6, 9, 10, 11, 12], 1:6])
coef.index = ['cost', 'vitamins', 'minerals', 'protein', 'calories']
coef.columns = DV_indexes
coef

,seeds,raisins,flakes,pecans,walnuts
cost,4,5,3,7,6
vitamins,10,20,10,30,20
minerals,5,7,4,9,2
protein,1,4,10,2,1
calories,500,450,160,300,500


In [37]:
type(coef)

pandas.core.frame.DataFrame

The next cell reads in the RHS of the constraints into their own dataframe.

In [38]:
rhs = pd.DataFrame(raw_data.iloc[9:13, 8])
rhs.index = list(coef.index[1:])
rhs.columns = ['rhs']
rhs

,rhs
vitamins,16
minerals,10
protein,15
calories,600


## Create pyomo model, decision variables, objective function, and constraints

In this section you will instantiate the decision variables, objective function and constraints for the Pyomo model.

First we will instantiate a ConcreteModel and store it in the variable `model`. This assumes we have already run `import pyomo.environ as pe`.

In [39]:
model = pe.ConcreteModel()

In [40]:
model.x.pprint()

AttributeError: 'ConcreteModel' object has no attribute 'x'

#### Define the Decision Variables

Create a pyomo variable named `x` with domain of nonnegative of real numbers.  Make sure you "attach" this variable to the `model` object. We will use the decision variable names that we already created in `DV_indexes` above.

In [41]:
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals)

To double check what you entered, use '.pprint()'.

In [42]:
model.x.pprint()

x : Size=5, Index=x_index
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
     flakes :     0 :  None :  None : False :  True : NonNegativeReals
     pecans :     0 :  None :  None : False :  True : NonNegativeReals
    raisins :     0 :  None :  None : False :  True : NonNegativeReals
      seeds :     0 :  None :  None : False :  True : NonNegativeReals
    walnuts :     0 :  None :  None : False :  True : NonNegativeReals


#### Define the objective function

The next cell defines the objective function `obj` using the data in the `coef` dataframe.  The expression argument, `expr=...`, shows a shorthand way to represent a sumproduct using a "list comprehension". The sense=pe.minimize means to minimize the objective function. Note you will find lots of code with the `sense=-1` argument which also means to maximize (`sense=1` is minimization, the default).

In [43]:
list_comp = [coef.loc['cost', i]*model.x[i] for i in DV_indexes]

In [45]:
type(list_comp)
list_comp

In [46]:
model.obj = pe.Objective(expr = sum([coef.loc['cost', i]*model.x[i] for i in DV_indexes]), 
                         sense = pe.minimize)

To double check what you entered, use 'pprint'.

In [48]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 4*x[seeds] + 5*x[raisins] + 3*x[flakes] + 7*x[pecans] + 6*x[walnuts]


#### Define the constraints

Next we need to define the four constraints using the data in the `coef` and `rhs` dataframes.  Make sure to "attach" each constraint to the `model` object. The constraints use a list comprehension to calculate the LHS sumproduct. 

In [49]:
model.cons_vit = pe.Constraint(expr=sum([coef.loc['vitamins', i]*model.x[i] 
                                         for i in DV_indexes]) >= rhs.loc['vitamins', 'rhs'])
model.cons_min = pe.Constraint(expr=sum([coef.loc['minerals', i]*model.x[i] 
                                         for i in DV_indexes]) >= rhs.loc['minerals', 'rhs'])
model.cons_prot = pe.Constraint(expr=sum([coef.loc['protein', i]*model.x[i]
                                          for i in DV_indexes]) >= rhs.loc['protein', 'rhs'])
model.cons_cal = pe.Constraint(expr=sum([coef.loc['calories', i]*model.x[i] 
                                          for i in DV_indexes]) >= rhs.loc['calories', 'rhs'])

Check the constraints. __NOTE:__ Lower and Upper shows the allowable range. The Upper is the RHS which is +Inf for all of these >= RHS constraints. Note for the first one for vitamins has Lower=16 and Upper=+Inf.

For a final check, let's print the entire model (rather than each separate part obj, decision variables, constraints)

In [50]:
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'seeds', 'raisins', 'flakes', 'pecans', 'walnuts'}

1 Var Declarations
    x : Size=5, Index=x_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         flakes :     0 :  None :  None : False :  True : NonNegativeReals
         pecans :     0 :  None :  None : False :  True : NonNegativeReals
        raisins :     0 :  None :  None : False :  True : NonNegativeReals
          seeds :     0 :  None :  None : False :  True : NonNegativeReals
        walnuts :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*x[seeds] + 5*x[raisins] + 3*x[flakes] + 7*x[pecans] + 6*x[walnuts]

4 Constraint Declarations
    cons_cal : Size=1, Index=None, Active=True
        K

#### Solve the model

Run the solver and examine the solution. You want to see "OPTIMAL LP Solution Found"

In [57]:
model.cons_prot.slack()

3.907985046680551e-14

In [52]:
opt = pe.SolverFactory('glpk')
success = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpy5emxmfh.glpk.raw
 --wglp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp1arscwqq.glpk.glp
 --cpxlp /var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpcntkbis5.pyomo.lp
Reading problem data from '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmpcntkbis5.pyomo.lp'...
5 rows, 6 columns, 21 non-zeros
54 lines were read
Writing problem data to '/var/folders/p8/y_pk7h153tld8fgnq6fz274r0000gn/T/tmp1arscwqq.glpk.glp'...
45 lines were written
GLPK Simplex Optimizer, v4.65
5 rows, 6 columns, 21 non-zeros
Preprocessing...
4 rows, 5 columns, 20 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  5.000e+02  ratio =  5.000e+02
GM: min|aij| =  4.229e-01  max|aij| =  2.364e+00  ratio =  5.590e+00
EQ: min|aij| =  1.789e-01  max|aij| =  1.000e+00  ratio =  5.590e+00
Constructing initial basis...
Size of triangular part is 4
      0: obj =   0.000000

This next cell will help you determine whether the solver found a solution or had an error (like poorly defined constraints, or infeasibility).  Pay particular attention to the "Solver" part of the output.

In [53]:
print(success)


Problem: 
- Name: unknown
  Lower bound: 7.53579952267303
  Upper bound: 7.53579952267303
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of nonzeros: 21
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.017139196395874023
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



#### View the Final Model Obj Function and Dec Variables

The next cell will extract the objective function and optimal solution (assuming the solver found a solution).  

In [54]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

DV = []  # create an empty list to store decision variables
for index in DV_indexes:
    DV.append(round(model.x[index].value, 3))
pd.DataFrame({'DV':DV_indexes,
             'Value':DV})

optimal objective value minimum cost = $7.54


,DV,Value
0,seeds,0.477
1,raisins,0.334
2,flakes,1.319
3,pecans,0.000
4,walnuts,0.000


#### View the slack

Print the slack in each constraint.   __NOTE:__ that ?.? e-13 is basically 0.

In [55]:
print(f'Vitamins slack = {model.cons_vit.slack():.2f}')
print(f'Minerals slack = {model.cons_min.slack():.2f}')
print(f'Protein slack = {model.cons_prot.slack():.2f}')
print(f'Calories slack = {model.cons_cal.slack():.2f}')

Vitamins slack = 8.64
Minerals slack = 0.00
Protein slack = 0.00
Calories slack = 0.00


### View the final LHS values

Also examine the final values for the LHS for each constraint. The "Body" is the final value.

In [56]:
model.cons_vit.display()
model.cons_min.display()
model.cons_prot.display()
model.cons_cal.display()

cons_vit : Size=1
    Key  : Lower : Body              : Upper
    None :  16.0 : 24.64200477326973 :  None
cons_min : Size=1
    Key  : Lower : Body               : Upper
    None :  10.0 : 10.000000000000016 :  None
cons_prot : Size=1
    Key  : Lower : Body               : Upper
    None :  15.0 : 15.000000000000039 :  None
cons_cal : Size=1
    Key  : Lower : Body              : Upper
    None : 600.0 : 600.0000000000007 :  None
